In [3]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [5]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")


Using cuda device


In [6]:
class NerualNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10),
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

In [7]:
model = NerualNetwork().to(device)
print(model)

NerualNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [13]:
X = torch.rand(1,28,28, device = device)
logits = model(X)
pred_probab =nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([2], device='cuda:0')


In [16]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [22]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [25]:
layer1 = nn.Linear(in_features = 28*28, out_features = 20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [26]:
print(f"Befor ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Befor ReLU: tensor([[-0.4233, -0.0761, -0.5026,  0.7623,  0.1720,  0.3051,  0.1715, -0.3988,
          0.7220,  0.5522, -0.7893, -0.8452, -0.0169,  0.4765, -0.0807, -0.2135,
         -0.4945, -0.0876, -0.1696, -0.1240],
        [-0.3861, -0.1386, -0.5523,  0.6510, -0.2866,  0.2672, -0.0420,  0.1002,
          0.8004,  0.0558, -0.6169, -0.6158, -0.1970,  0.1049, -0.1700, -0.2286,
         -0.8116,  0.2174, -0.2313, -0.4999],
        [-0.2048, -0.0654,  0.0222,  0.6851,  0.0596,  0.1968,  0.2426,  0.0060,
          0.6617,  0.0080, -0.6167, -0.6668,  0.2736, -0.1586, -0.2472, -0.0094,
         -0.2931,  0.3203, -0.2499, -0.4525]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.7623, 0.1720, 0.3051, 0.1715, 0.0000, 0.7220,
         0.5522, 0.0000, 0.0000, 0.0000, 0.4765, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.6510, 0.0000, 0.2672, 0.0000, 0.1002, 0.8004,
         0.0558, 0.0000, 0.0000, 0.0000, 0.1049, 0.000

In [27]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [28]:
softmax = nn.Softmax(dim = 1)
pred_probab = softmax(logits)

In [31]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
  print(f"Layer: {name} | Size {param.size()} | Values {param[:2]} ")

Model structure: NerualNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size torch.Size([512, 784]) | Values tensor([[ 0.0062,  0.0029,  0.0005,  ...,  0.0014, -0.0289, -0.0340],
        [-0.0203, -0.0137,  0.0354,  ...,  0.0058, -0.0053, -0.0104]],
       device='cuda:0', grad_fn=<SliceBackward0>) 
Layer: linear_relu_stack.0.bias | Size torch.Size([512]) | Values tensor([0.0017, 0.0236], device='cuda:0', grad_fn=<SliceBackward0>) 
Layer: linear_relu_stack.2.weight | Size torch.Size([512, 512]) | Values tensor([[-0.0131,  0.0256,  0.0321,  ...,  0.0196,  0.0120, -0.0190],
        [ 0.0284, -0.0145,  0.0405,  ..., -0.0232,  0.0174, -0.0415]],
       device='cuda:0', grad_fn=<SliceBackward0>